This notebook is used to see if we can find any information for the wind speeds contained at height (depth) 0

In [34]:
import numpy as np
import pandas as pd
import xarray as xr
import os
import math
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm
from functools import reduce
import operator
import time
import itertools
import IPython
from collections import defaultdict
#Enables the line profiler magic command #lprun
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [ ]:
#Get the filenames for bouys with depth 0

#Read result_df with pickle
read_folder = './model_bouy_comparison_era5/'
result_df_fn = 'result_dfs'

with open(os.path.join(read_folder, result_df_fn),'rb') as f_r:
    result_df_sel, result_df_interp = pickle.load(f_r)

display(result_df_sel)

In [24]:
bouy_data_dir = '/data/exjobb/sarssw/bouy/INSITU_GLO_PHYBGCWAV_DISCRETE_MYNRT_013_030/MO'
bouy_0_depth_list = result_df_sel[
    (result_df_sel['bouy_depth'] == 0) &
    (result_df_sel['bouy_variable_name'] == 'WSPD')
    ]['bouy_file_name'].unique()

print(len(bouy_0_depth_list), ' Files')
display(bouy_0_depth_list)

55  Files


array(['GL_TS_MO_2200183.nc', 'GL_TS_MO_2200104.nc',
       'GL_TS_MO_2200188.nc', 'GL_TS_MO_2200189.nc',
       'GL_TS_MO_2100229.nc', 'GL_TS_MO_2200101.nc',
       'GL_TS_MO_2200298.nc', 'NO_TS_MO_6200145.nc',
       'GL_TS_MO_2300451.nc', 'GL_TS_MO_2200108.nc',
       'BS_TS_MO_SPOT0772.nc', 'NO_TS_MO_6200165.nc',
       'BS_TS_MO_SPOT0773.nc', 'GL_TS_MO_6100002.nc',
       'NO_TS_MO_6300110.nc', 'NO_TS_MO_6200170.nc',
       'NO_TS_MO_6200114.nc', 'NO_TS_MO_6300112.nc',
       'GL_TS_MO_6100001.nc', 'NO_TS_MO_6200150.nc',
       'NO_TS_MO_6200149.nc', 'GL_TS_MO_2200191.nc',
       'GL_TS_MO_2300492.nc', 'NO_TS_MO_6200121.nc',
       'GL_TS_MO_2200190.nc', 'GL_TS_MO_2200192.nc',
       'GL_TS_MO_2300099.nc', 'BS_TS_MO_SPOT0776.nc',
       'GL_TS_MO_2300453.nc', 'GL_TS_MO_2300454.nc',
       'GL_TS_MO_2300091.nc', 'GL_TS_MO_2200103.nc',
       'NO_TS_MO_6300117.nc', 'BO_TS_MO_Koster.nc', 'GL_TS_MO_2200106.nc',
       'NO_TS_MO_6200144.nc', 'GL_TS_MO_2200186.nc',
       'GL_TS_MO_2200

In [25]:
#Manual analysis
for file in bouy_0_depth_list:
    ds = xr.open_dataset(os.path.join(bouy_data_dir, file))

    IPython.display.clear_output()
    print(file)
    display(ds)

    if input("type q to quit, leave blank to continue") in ['q', 'Q', 'exit', 'quit', 'stop']:
        break


GL_TS_MO_2200183.nc


<xarray.Dataset>
Dimensions:       (TIME: 81639, LATITUDE: 81639, LONGITUDE: 81639,
                   POSITION: 81639, DEPTH: 3)
Coordinates:
  * TIME          (TIME) datetime64[ns] 2013-06-21T05:00:00 ... 2023-01-31T23...
  * LATITUDE      (LATITUDE) float32 34.73 34.73 34.73 ... 34.73 34.73 34.73
  * LONGITUDE     (LONGITUDE) float32 126.2 126.2 126.2 ... 126.2 126.2 126.2
Dimensions without coordinates: POSITION, DEPTH
Data variables: (12/21)
    TIME_QC       (TIME) float32 ...
    POSITION_QC   (POSITION) float32 ...
    DC_REFERENCE  (TIME) object ...
    DEPH          (TIME, DEPTH) float32 ...
    DEPH_QC       (TIME, DEPTH) float32 ...
    TEMP          (TIME, DEPTH) float64 ...
    ...            ...
    VGTA          (TIME, DEPTH) float64 ...
    VGTA_QC       (TIME, DEPTH) float32 ...
    WDIR          (TIME, DEPTH) float64 ...
    WDIR_QC       (TIME, DEPTH) float32 ...
    WSPD          (TIME, DEPTH) float64 ...
    WSPD_QC       (TIME, DEPTH) float32 ...
Attributes: (12/49)
    data_type:                      OceanSITES time-series data
    format_version:                 1.4
    platform_code:                  2200183
    institution:                    Korea Meteorological Administration (KMA)
    institution_edmo_code:          4635
    site_code:                       
    ...                             ...
    last_date_observation:          2023-01-31T23:00:00Z
    last_latitude_observation:      34.73000
    last_longitude_observation:     126.24000
    date_update:                    2023-02-07T19:50:00Z
    history:                        2023-02-07T19:50:00Z : Creation
    data_mode:                      R

In [51]:
#analyse attributes
attr_dict = {
    'global':defaultdict(list),
    'deph':defaultdict(list),
    'deph_qc':defaultdict(list),
}

for file in bouy_0_depth_list:
    ds = xr.open_dataset(os.path.join(bouy_data_dir, file))

    global_attr = ds.attrs.keys()
    for attr in global_attr:
        attr_dict['global'][attr].append(file)

    deph_attr = ds['DEPH'].attrs.keys()
    for attr in deph_attr:
        attr_dict['deph'][attr].append(file)

    deph_qc_attr = ds['DEPH_QC'].attrs.keys()
    for attr in deph_qc_attr:
        attr_dict['deph_qc'][attr].append(file)


In [52]:
attr_count_dict = {k:{kk:len(vv) for kk,vv in v.items()} for k,v in attr_count_dict.items()}
display(attr_count_dict)

{'global': {'data_type': 55,
  'format_version': 55,
  'platform_code': 55,
  'institution': 55,
  'institution_edmo_code': 55,
  'site_code': 55,
  'wmo_platform_code': 55,
  'coriolis_platform_code': 32,
  'platform_name': 55,
  'ices_platform_code': 55,
  'source': 55,
  'source_platform_category_code': 55,
  'wmo_inst_type': 55,
  'references': 55,
  'comment': 55,
  'Conventions': 55,
  'netcdf_version': 55,
  'title': 55,
  'summary': 55,
  'naming_authority': 55,
  'author': 37,
  'id': 55,
  'cdm_data_type': 55,
  'area': 55,
  'bottom_depth': 55,
  'institution_references': 55,
  'contact': 55,
  'data_assembly_center': 55,
  'pi_name': 55,
  'distribution_statement': 55,
  'citation': 55,
  'update_interval': 55,
  'qc_manual': 55,
  'doi': 55,
  'license': 31,
  'geospatial_lat_min': 55,
  'geospatial_lat_max': 55,
  'geospatial_lon_min': 55,
  'geospatial_lon_max': 55,
  'geospatial_vertical_min': 55,
  'geospatial_vertical_max': 55,
  'time_coverage_start': 55,
  'time_cov

In [54]:
[{var:attr_dict['global'][var]} for var in ['quality_control_indicator', 'quality_index', 'history_version']]

[{'quality_control_indicator': ['BS_TS_MO_SPOT0772.nc',
   'BS_TS_MO_SPOT0773.nc',
   'BS_TS_MO_SPOT0776.nc',
   'BO_TS_MO_Koster.nc',
   'BO_TS_MO_Norrbyn.nc',
   'BO_TS_MO_Asko.nc',
   'BO_TS_MO_HuvudskarOst.nc']},
 {'quality_index': ['BS_TS_MO_SPOT0772.nc',
   'BS_TS_MO_SPOT0773.nc',
   'BS_TS_MO_SPOT0776.nc',
   'BO_TS_MO_Koster.nc',
   'BO_TS_MO_Norrbyn.nc',
   'BO_TS_MO_Asko.nc',
   'BO_TS_MO_HuvudskarOst.nc']},
 {'history_version': ['BO_TS_MO_Koster.nc',
   'BO_TS_MO_Norrbyn.nc',
   'BO_TS_MO_Asko.nc',
   'BO_TS_MO_HuvudskarOst.nc']}]

In [55]:
display(xr.open_dataset(os.path.join(bouy_data_dir, 'BO_TS_MO_HuvudskarOst.nc')))

<xarray.Dataset>
Dimensions:      (TIME: 93816, LATITUDE: 93816, LONGITUDE: 93816,
                  POSITION: 93816, DEPTH: 31)
Coordinates:
  * TIME         (TIME) datetime64[ns] 2001-05-10T22:00:00 ... 2022-11-13T13:...
  * LATITUDE     (LATITUDE) float32 65.0 65.0 65.0 65.0 ... 58.94 58.94 58.94
  * LONGITUDE    (LONGITUDE) float32 12.0 12.0 12.0 12.0 ... 19.16 19.16 19.16
Dimensions without coordinates: POSITION, DEPTH
Data variables: (12/47)
    TIME_QC      (TIME) float32 ...
    POSITION_QC  (POSITION) float32 ...
    VHM0         (TIME, DEPTH) float64 ...
    VHM0_QC      (TIME, DEPTH) float32 ...
    VHM0_DM      (TIME, DEPTH) object ...
    TUR4         (TIME, DEPTH) float64 ...
    ...           ...
    HCDT         (TIME, DEPTH) float64 ...
    HCDT_QC      (TIME, DEPTH) float32 ...
    TUR2         (TIME, DEPTH) float64 ...
    TUR2_QC      (TIME, DEPTH) float32 ...
    HCSP         (TIME, DEPTH) float64 ...
    HCSP_QC      (TIME, DEPTH) float32 ...
Attributes: (12/50)
    contact:                        cmems-service@smhi.se
    quality_index:                  A
    netcdf_version:                 netCDF-4 classic model
    Conventions:                    CF-1.6 Copernicus-InSituTAC-FormatManual-...
    naming_authority:               Copernicus Marine In Situ
    wmo_inst_type:                  
    ...                             ...
    geospatial_lat_min:             54.0000
    geospatial_lon_max:             24.9998
    institution_edmo_code:          545
    title:                          Global Ocean - In Situ Wave Observation R...
    geospatial_lat_max:             59.7653
    history:                        2020-05-12T19:05:04Z File created by SMHI...